In [7]:
### imports
import json

from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))

In [2]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
medDocs[262]

{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [3]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
        cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        tokens += sentTokens2
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = medDocs
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])

    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

In [4]:
print(len(cleanedDocs))
cleanedDocs[262]

263


{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': ['recurrent',
  'invasive',
  'ductal',
  'breast',
  'carcinoma',
  'presenting',
  'primary',
  'adrenal',
  'insufficiency',
  'adrenal',
  'crisis'],
 'caption': ['histologic',
  'section',
  'adrenal',
  'metastatic',
  'disease',
  'tumor',
  'cells',
  'arranged',
  'solid',
  'nests',
  'infiltrating',
  'lymph',
  'node',
  'tissue']}

In [5]:
# constructing inverted index
def getInvertedIdx(medDocs):
    invertedIdx = {}

    for i, doc in enumerate(medDocs):
        for term in doc['articleTitle']:
            if term in invertedIdx:
                invertedIdx[term].add((i,'t'))
            else:
                invertedIdx[term] = {(i,'t')}
        for term in doc['caption']:
            if term in invertedIdx:
                invertedIdx[term].add((i,'c'))
            else:
                invertedIdx[term] = {(i,'c')}
    return invertedIdx
           
titleInvIdx = getInvertedIdx(cleanedDocs)
titleInvIdx


{'novel': {(0, 't'),
  (1, 't'),
  (16, 't'),
  (22, 't'),
  (23, 't'),
  (24, 't'),
  (25, 't'),
  (26, 't'),
  (27, 't'),
  (96, 't'),
  (97, 't'),
  (98, 't'),
  (99, 't'),
  (100, 't'),
  (102, 't'),
  (103, 't'),
  (106, 't'),
  (133, 't'),
  (134, 't'),
  (135, 't'),
  (136, 't'),
  (137, 't'),
  (138, 't'),
  (139, 't'),
  (140, 't'),
  (145, 'c'),
  (199, 't'),
  (200, 't'),
  (201, 't'),
  (202, 't'),
  (252, 't'),
  (253, 't')},
 'use': {(0, 't'),
  (1, 't'),
  (141, 't'),
  (142, 'c'),
  (142, 't'),
  (143, 't'),
  (209, 'c')},
 'stereotactic': {(0, 'c'), (0, 't'), (1, 'c'), (1, 't')},
 'radiotherapy': {(0, 'c'), (0, 't'), (1, 'c'), (1, 't')},
 'remnant': {(0, 't'), (1, 't')},
 'adrenal': {(0, 'c'),
  (0, 't'),
  (1, 't'),
  (28, 't'),
  (29, 'c'),
  (29, 't'),
  (30, 'c'),
  (30, 't'),
  (247, 'c'),
  (248, 'c'),
  (249, 'c'),
  (250, 'c'),
  (257, 'c'),
  (257, 't'),
  (258, 't'),
  (259, 't'),
  (260, 't'),
  (261, 'c'),
  (261, 't'),
  (262, 'c'),
  (262, 't')},
 'tissue

In [6]:
postingList = invertedIdx['ball']
postingList

NameError: name 'invertedIdx' is not defined

In [ ]:
# Boolean operations:

In [ ]:
def orPostings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    n1 = len(posting1)
    n2 = len(posting2)
    while p1 < n1 and p2 <n2:
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] >posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < n1:
        result.append(posting1[p1])
        p1 += 1
    while p2 < n2:
        result.append(posting2[p2])
        p2 += 1
    return result

In [ ]:

pl_1 = list(invertedIdx['soccer'])
pl_2 = list(invertedIdx['glider'])
orPostings(pl_1, pl_2)

In [ ]:
def andPostings(posting1, posting2):
    p1 = 0
    p2 = 0
    n1 = len(posting1)
    n2 = len(posting2)
    result = list()
    
    while p1 < n1 and p2 < n2:
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
            
    return result 
        

In [ ]:
pl_1 = list(invertedIdx['soccer'])
pl_2 = list(invertedIdx['ball'])
andPostings(pl_1, pl_2)